# Exercise 05:  Solving differential equation-based Bayesian inverse problems using CUQIpy

Here we build a Bayesian problem in which the forward model is a differential equation model, 1D Poisson problem in particular.

## Learning objectives of this notebook:
- Define differential equation model in cuqi.
- Use basis functions (KL-expansion) to represent the parameter space.
- Introducing mapped geometries

## TOC: 
* [PDE model](#PDE_model)
* [The Bayesian inverse problem](#inverse_problem)


We first import the required python standard packages

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

From cuqi package, we import the classes that we use in this exercise

In [ ]:
from cuqi.geometry import Continuous1D, MappedGeometry, KLExpansion
from cuqi.pde import SteadyStateLinearPDE
from cuqi.model import PDEModel
from cuqi.distribution import GaussianCov, Posterior
from cuqi.sampler import CWMH, NUTS, pCN 

## PDE model <a class="anchor" id="PDE_model"></a> 

The PDE problem we consider here is a Poisson problem with heterogeneous coefficient $\kappa(x)$:
$$ \nabla \cdot (\kappa(x) \nabla u(x)) = f(x) \in \Omega,$$
where $u(x)$ is the solution, and $f(x)$ is the source term. This equation, for example, can model a steady-state head conductivity problem where $\kappa(x)$ is the thermal diffusivity and $u(x)$ is the temperature. $f(x)$ can be a heat source or sink.

We assume the domain $\Omega$ is the 1D interval $[0,L]$, where $L$ is the domain length. We can think of this as heat conductivity in a rod. In the 1D case, the PDE system redcues to an ODE of the form
$$ \frac{d}{dx} \left(\kappa(x) \frac{d}{dx} u(x)\right) = f(x) \in [0,L],$$

We discretize the differential operator $ \frac{d}{dx} \left(\kappa(x) \frac{d}{dx} . \right)$ using finite difference method (forward difference discretization in particular). The result discretized system is of the form 

$$\mathbf{A}(\pmb{\kappa})\mathbf{u} = \mathbf{f}$$

where $\pmb{\kappa}$, $\mathbf{u}$, and $\mathbf{f}$ are the discretized thermal conductivity, discretized temperature, and the discretized source term respectively.

### Method to build the forward finite difference operator $\frac{d}{dx}$

In [ ]:
def build_Dx(N,dx):
    Dx = - np.diag(np.ones(N), 0) + np.diag(np.ones(N-1), 1) 
    vec = np.zeros(N)
    vec[0] = 1
    Dx = np.concatenate([vec.reshape([1,-1]), Dx], axis=0)
    Dx /= dx 
    return Dx


### Build the PDE form

In [ ]:
dim = 51 # Number of finite difference nodes
N = dim-1 # Number of solution nodes
L = 1 # Length of the 1D domain
dx = 1./N   # grid spacing 

#Source term
grid = np.linspace(dx, L, N, endpoint=False)
source = lambda xs: 10*np.exp( -( (xs - 0.5)**2 ) / 0.02) 
rhs = source(grid)

# forward finite difference operator 
Dx = build_Dx(N,dx)

# PDE form: LHS(x)u=rhs(x) # x is the Bayesian problem parameter 
PDE_form = lambda x: (Dx.T @ np.diag(x) @ Dx, rhs)
PDE = SteadyStateLinearPDE(PDE_form)

## The Bayesian inverse problem <a class="anchor" id="inverse_problem"></a>
We want to quantify the uncertainty in the thermal conductivity $\kappa$ given the measurement of the temperature $u$ everywhere in the domain. We can denote the forward model as $\mathcal{G}(\kappa) = u$. The data is then given by:

$$ d = \mathcal{G}(\kappa) + \eta$$


where $\eta$ is the measurement noise. We assume both the prior distribution of $\kappa$ and the noise distribution are Gaussian.  

### Build the CUQI forward model: domain and range geometries


In [ ]:
grid_domain = np.linspace(0, L, dim, endpoint=True)
grid_range  = np.linspace(1./N, L, N, endpoint=False)
domain_geometry = Continuous1D(grid_domain)
# op1
#domain_geometry = MappedGeometry(Continuous1D(grid_domain), map = lambda x: np.exp(x))
range_geometry = Continuous1D(grid_range)

### Build the CUQI forward model: the PDE model

In [ ]:
model = PDEModel(PDE,range_geometry,domain_geometry)

### Set up an exact solution

In [ ]:
x_exact = np.exp( 5*grid_domain*np.exp(-2*grid_domain)*np.sin(L-grid_domain) )

### Generate data

In [ ]:
# Generate exact data
b_exact = model.forward(x_exact,is_par=False)

# Add noise to data
SNR = 200
sigma = np.linalg.norm(b_exact)/SNR
sigma2 = sigma*sigma # variance of the observation Gaussian noise
data = b_exact + np.random.normal( 0, sigma, b_exact.shape )

### Formulate the posterior distribution

In [ ]:
likelihood = GaussianCov(model, sigma2*np.eye(range_geometry.dim))
prior = GaussianCov(np.zeros(domain_geometry.dim)+1, .4)
posterior = Posterior(likelihood,prior,data)

### Sample the posterior distribution

In [ ]:
mySampler = CWMH(posterior)
samples,o2,o3 = mySampler.sample(100,10)
samples.geometry = domain_geometry
samples.plot_ci(95,exact=x_exact)